In [1]:
import numpy as np
import cv2
# import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib tk
# qt doesn"t work!

In [2]:
# Camera calibration
## Here I use the code based on ../examples/example.ipynb and make some modifications
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane

# Make a list of calibration images
# images = glob.glob('/Users/Victoria/udacity-git-course/CarND/LaneLines/CarND-Advanced-Lane-Lines/camera_cal/calibration*.jpg')
## Instead of glob, I prefer to get the image list
## Lesson: use the ABSOLUTE path!
path = '/Users/Victoria/udacity-git-course/CarND/LaneLines/CarND-Advanced-Lane-Lines'
images = []
for i in range(1,21):
    img_path = path + '/camera_cal/calibration' + str(i) + '.jpg'
    img = cv2.imread(img_path)
    images.append(img)

# Step through the list and search for chessboard corners
for img in images:

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

# Get camera matrix and distortion coefficients
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, images[0].shape[1::-1], None, None)

In [ ]:
# Test camera calibration
dst = cv2.undistort(images[1], mtx, dist, None, mtx)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Distorted')
ax1.imshow(images[1], cmap='gray')
ax2.set_title('Undistorted')
ax2.imshow(dst, cmap='gray')
#plt.savefig(path + '/output_images/undistorted.png')

In [3]:
# Get undistorted images
# Note: when using cv2.imwrite, better to use cv2.imread, so that the images look normal
# images = []
# images.append(cv2.imread(path + '/test_images/test1.jpg'))
# images.append(cv2.imread(path + '/test_images/test2.jpg'))
# images.append(cv2.imread(path + '/test_images/test3.jpg'))
# images.append(cv2.imread(path + '/test_images/test4.jpg'))
# images.append(cv2.imread(path + '/test_images/test5.jpg'))
# images.append(cv2.imread(path + '/test_images/test6.jpg'))
# images.append(cv2.imread(path + '/test_images/straight_lines1.jpg'))
# images.append(cv2.imread(path + '/test_images/straight_lines2.jpg'))

# for i in range(8):
#     dst = cv2.undistort(images[i], mtx, dist, None, mtx)
#     cv2.imwrite(path+'/undistorted_images/'+str(i)+'.jpg', dst)

# Get RGB images instead of BGR, because BGR paints the yellow lines in blue
path = '/Users/Victoria/udacity-git-course/CarND/LaneLines/CarND-Advanced-Lane-Lines'
undistorted = []
for i in range(8):
    undistorted.append(mpimg.imread(path+'/undistorted_images/'+str(i)+'.jpg'))

In [4]:
# Thresholding function
def thresh(img):
    # Sobel function
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    kernel_size=15
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, kernel_size)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, kernel_size)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    mag = np.sqrt(sobelx**2 + sobely**2) # magnitude of gradients
    direc = np.arctan2(abs_sobely, abs_sobelx) # direction of gradients

    # Convert the absolute value image to 8-bit
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    scaled_mag = np.uint8(255*mag/np.max(mag))

    sobel_thresh=(20,255)
    sxbinary = np.zeros_like(scaled_sobelx)
    sxbinary[(scaled_sobelx >= sobel_thresh[0]) & (scaled_sobelx <= sobel_thresh[1])] = 1
    sybinary = np.zeros_like(scaled_sobely)
    sybinary[(scaled_sobely >= sobel_thresh[0]) & (scaled_sobely <= sobel_thresh[1])] = 1

    mag_thresh=(10,255)
    mag_binary = np.zeros_like(mag)
    mag_binary[(scaled_mag >= mag_thresh[0]) & (scaled_mag <= mag_thresh[1])] = 1

    direc_thresh=(0.7,1.3)
    direc_binary = np.zeros_like(direc)
    direc_binary[(direc >= direc_thresh[0]) & (direc <= direc_thresh[1])] = 1

    # HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    S = hls[:,:,2]
    S_thresh=(90,255)
    S_binary = np.zeros_like(S)
    S_binary[(S > S_thresh[0]) & (S <= S_thresh[1])] = 1
    
    # RGB color space
    R = img[:,:,0]
    R_thresh=(20,255)
    R_binary = np.zeros_like(R)
    R_binary[(R > R_thresh[0]) & (R <= R_thresh[1])] = 1

    # Combine all conditions
    combined = np.zeros_like(S)
    combined[((sxbinary == 1) & (sybinary == 1)) | ((mag_binary == 1) & (direc_binary == 1)) & ((S_binary == 1) & (R_binary == 1))] = 1
    
    return combined

In [5]:
# Test the thresholding function
thresh_images = []
for i in range(8):
    img_thresh = thresh(undistorted[i])
    thresh_images.append(img_thresh)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Original')
    ax1.imshow(undistorted[i], cmap='gray')
    ax2.set_title('Thresholding')
    ax2.imshow(img_thresh, cmap='gray')
    #plt.savefig(path + '/output_images/threshold'+str(i)+'.jpg')

In [6]:
# Perspective transform function
src = np.float32(
    [[220,700],
     [578,460],
     [705,460],
     [1100,700]])
dst = np.float32(
    [[270,720],
     [270,0],
     [1010,0],
     [1010,720]])

def warp(img, src, dst):
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, img_size, flags = cv2.INTER_LINEAR)

    return warped

In [7]:
# Test perspective transformation on warped image
binary_warped = [] # store the warped images

for i in range(8):
    
    # convert binary image to color in order to draw red lines
    img_thresh_rgb = np.zeros((thresh_images[i].shape[0],thresh_images[i].shape[1],3), dtype="uint8")
    img_thresh_rgb[:,:,0] = thresh_images[i] * 255
    img_thresh_rgb[:,:,1] = thresh_images[i] * 255
    img_thresh_rgb[:,:,2] = thresh_images[i] * 255

    warped = warp(thresh_images[i], src, dst)
    binary_warped.append(warped)
    warped_rgb = np.zeros((warped.shape[0],warped.shape[1],3), dtype="uint8")
    warped_rgb[:,:,0] = warped * 255
    warped_rgb[:,:,1] = warped * 255
    warped_rgb[:,:,2] = warped * 255

    # draw source points on image
    img_copy = np.copy(img_thresh_rgb)*0
    for j in range(-1,3):
        cv2.line(img_copy,(src[j][0], src[j][1]),(src[j+1][0], src[j+1][1]), (255,0,0), 5)
    combo_src = cv2.addWeighted(img_thresh_rgb, 1, img_copy, 0.5, 0)

    # draw destination points on image
    warped_copy = np.copy(warped_rgb)*0
    for j in range(-1,3):
        cv2.line(warped_copy,(dst[j][0], dst[j][1]),(dst[j+1][0], dst[j+1][1]), (255,0,0), 5)
    combo_warped = cv2.addWeighted(warped_rgb, 1, warped_copy, 0.5, 0)

    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.set_title('Source image')
    ax1.imshow(combo_src, cmap='gray')
    ax2.set_title('Destination image')
    ax2.imshow(combo_warped, cmap='gray')
    #plt.savefig(path + '/output_images/warped'+str(i)+'.jpg')

<ipython-input-7-355edb905656>:22: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.line(img_copy,(src[j][0], src[j][1]),(src[j+1][0], src[j+1][1]), (255,0,0), 5)
<ipython-input-7-355edb905656>:28: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.line(warped_copy,(dst[j][0], dst[j][1]),(dst[j+1][0], dst[j+1][1]), (255,0,0), 5)


In [8]:
# Histogram, sliding window and curvature calculation function. Code from Lesson 8.
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

            
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')

    return out_img, left_fit, right_fit


def measure_curvature_pixels(binary_warped):
    '''
    Calculates the curvature of polynomial functions in pixels.
    '''
    # Start by generating our fake example data
    # Make sure to feed in your real data instead in your project!
    out_img, left_fit, right_fit = fit_polynomial(binary_warped)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 720
    
    # Calculation of R_curve (radius of curvature)
    left_curverad = (1+(2*left_fit[0]*y_eval+left_fit[1])**2)**(3/2)/abs(2*left_fit[0])
    right_curverad = (1+(2*right_fit[0]*y_eval+right_fit[1])**2)**(3/2)/abs(2*right_fit[0])
    
    return left_curverad, right_curverad


def measure_curvature_real(binary_warped):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = 720
    
    # Implement the calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad

In [ ]:
# Test histogram and sliding window function
for i in range(8):
    plt.close()
    out_img, left_fit, right_fit = fit_polynomial(binary_warped[i])
    plt.imshow(out_img)
    plt.savefig(path+'/output_images/histogram'+str(i)+'.jpg')

In [ ]:
# Test curvation calculation
curv_pixel = []
for img in binary_warped:
    left_curverad, right_curverad = measure_curvature_pixels(img)
    curv_pixel.append([left_curverad, right_curverad])
print(curv_pixel)

curv_real = []
for img in binary_warped:
    left_curverad, right_curverad = measure_curvature_real(img)
    curv_real.append([left_curverad, right_curverad])
print(curv_real)

In [9]:
# Offset function.
def offset_left_of_center(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    mid_lane = (leftx_base + rightx_base) / 2
    xm_per_pix = 3.7/700
    offset = (midpoint - mid_lane) * xm_per_pix # offset in meters. offset<0 means left of center.
    
    return offset

In [ ]:
# Test offset
offsets = []
for img in binary_warped:
    offset = offset_left_of_center(img)
    offsets.append(offset)
print(offsets)

In [10]:
# Warp to original image function. Code from project instructions.
def warp_to_original(undist, binary_warped, src, dst):
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    out_img, left_fit, right_fit = fit_polynomial(binary_warped)
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    plt.close()
    cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    Minv = cv2.getPerspectiveTransform(dst, src)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0]))
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    return result

In [ ]:
# Test warp to original image
plt.close()
for i in range(8):
    result = warp_to_original(undistorted[i], binary_warped[i], src, dst)
    mpimg.imsave(path+'/output_images/warpToOri'+str(i)+'.jpg', result)
#    cv2.imwrite(path+'/output_images/warpToOri'+str(i)+'.jpg', result)
#     plt.imshow(result)
#     plt.savefig(path+'/output_images/warpToOri'+str(i)+'.jpg')

In [14]:
# Search from prior function. Code from Lesson 8.
def search_around_poly(binary_warped):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the area of search based on activated x-values within the +/- margin
    img, left_fit, right_fit = fit_polynomial(binary_warped)
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    ## End visualization steps ##
    
    return result

In [18]:
# Test search from prior function
for i in range(8):
    result = search_around_poly(binary_warped[i])
    plt.imshow(result)
    plt.savefig(path+'/output_images/searchAroundPoly'+str(i)+'.jpg')
    plt.close()